In [1]:
import json
import pandas as pd
import math
import numpy as np
import re
from sklearn.metrics import r2_score, mean_squared_error

with open('qol_fu_retrieval_async_all.json', 'r', encoding="utf-8") as f:
    response = json.load(f)

In [2]:
label = pd.read_csv("../fu_improve_label.csv")
label.rename(columns={'PKD': 'SPKD'}, inplace=True)
label

,PCS,MCS,SPKD,EKD,BKD
0,False,False,False,False,True
1,False,False,True,True,True
2,True,False,False,False,False
3,False,False,False,False,False
4,False,True,False,False,False
...,...,...,...,...,...
232,True,True,False,True,True
233,True,False,True,False,False
234,True,True,True,True,True
235,True,False,True,False,True


In [3]:
response = "To address the physician's questions regarding the improvement of KDQOL domain scores, we need to follow these steps:\n\n1. **Extract relevant items for each KDQOL domain.**\n2. **Map the responses to scores using the provided score mapping dictionary.**\n3. **Calculate the average score for each domain.**\n4. **Compare the scores from the first and second surveys to determine if there has been an improvement.**\n\nBelow is the Python code to achieve this:\n\n```python\nimport pandas as pd\nimport numpy as np\n\n# Load the DataFrame containing all the patients' responses\ndf = pd.read_csv(\"../eng_base_sub.csv\")  # no need to modify the path\n\n# Define the mapping of responses to scores\nscore_mapping = {\n    'Q1': {'Poor': 0.0, 'Fair': 25.0, 'Good': 50.0, 'Very good': 75.0, 'Excellent': 100.0},\n    'Q2': {'Much worse now than one year ago': 0.0, 'Somewhat worse now than one year ago': 25.0, 'About the same than one year ago': 50.0, 'Somewhat better now than one year ago': 75.0, 'Much better now than one year ago': 100.0},\n    'Q3': {'Yes, limited a lot': 0.0, 'Yes, limited a little': 50.0, 'No, not limited at all': 100.0},\n    'Q4 / Q5': {'Yes': 0.0, 'No': 100.0},\n    'Q6': {'Extremely': 0.0, 'Quite a bit': 25.0, 'Moderately': 50.0, 'Slightly': 75.0, 'Not at all': 100.0},\n    'Q7': {'Very severe': 0.0, 'Severe': 20.0, 'Moderate': 40.0, 'Mild': 60.0, 'Very mild': 80.0, 'None': 100.0},\n    'Q8': {'Extremely': 0.0, 'Quite a bit': 25.0, 'Moderately': 50.0, 'A little bit': 75.0, 'Not at all': 100.0},\n    'Q9 a,d,e,h': {'None of the time': 0.0, 'A little of the time': 20.0, 'Some of the time': 40.0, 'A good bit of the time': 60.0, 'Most of the time': 80.0, 'All of the time': 100.0},\n    'Q9 b,c,f,g,i': {'All of the time': 0.0, 'Most of the time': 20.0, 'A good bit of the time': 40.0, 'Some of the time': 60.0, 'A little of the time': 80.0, 'None of the time': 100.0},\n    'Q10': {'All of the time': 0.0, 'Most of the time': 25.0, 'Some of the time': 50.0, 'A little of the time': 75.0, 'None of the time': 100.0},\n    'Q11 a,c': {'Definitely true': 0.0, 'Mostly true': 25.0, 'Don\\'t know': 50.0, 'Mostly false': 75.0, 'Definitely false': 100.0},\n    'Q11 b,d': {'Definitely false': 0.0, 'Mostly false': 25.0, 'Don\\'t know': 50.0, 'Mostly true': 75.0, 'Definitely true': 100.0},\n    'Q12': {'Definitely true': 0.0, 'Mostly true': 25.0, 'Don\\'t know': 50.0, 'Mostly false': 75.0, 'Definitely false': 100.0},\n    'Q13 a-d, f': {'All of the time': 0.0, 'Most of the time': 20.0, 'A good bit of the time': 40.0, 'Some of the time': 60.0, 'A little of the time': 80.0, 'None of the time': 100.0},\n    'Q13 e': {'None of the time': 0.0, 'A little of the time': 20.0, 'Some of the time': 40.0, 'A good bit of the time': 60.0, 'Most of the time': 80.0, 'All of the time': 100.0},\n    'Q14': {'Extremely bothered': 0.0, 'Very much bothered': 25.0, 'Moderately bothered': 50.0, 'Somewhat bothered': 75.0, 'Not at all bothered': 100.0},\n    'Q15': {'Extremely bothered': 0.0, 'Very much bothered': 25.0, 'Moderately bothered': 50.0, 'Somewhat bothered': 75.0, 'Not at all bothered': 100.0},\n    'Q16': {'No': 0.0, 'Yes': 100.0},\n    'Q16 a,b': {'Severe problem': 0.0, 'Very much a problem': 25.0, 'Somewhat of a problem': 50.0, 'A little problem': 75.0, 'Not a problem': 100.0},\n    'Q18 a,c': {'All of the time': 0.0, 'Most of the time': 20.0, 'A good bit of the time': 40.0, 'Some of the time': 60.0, 'A little of the time': 80.0, 'None of the time': 100.0},\n    'Q18 b': {'None of the time': 0.0, 'A little of the time': 20.0, 'Some of the time': 40.0, 'A good bit of the time': 60.0, 'Most of the time': 80.0, 'All of the time': 100.0},\n    'Q19': {'Very dissatisfied': 0.0, 'Somewhat dissatisfied': 33.33, 'Somewhat satisfied': 66.67, 'Very satisfied': 100.0},\n    'Q20': {'No': 0.0, 'Yes': 100.0},\n    'Q21': {'Yes': 0.0, 'No': 100.0},\n    'Q23': {'Very poor': 0.0, 'Poor': 16.67, 'Fair': 33.33, 'Good': 50.0, 'Very good': 66.67, 'Excellent': 83.33, 'The best': 100.0},\n    'Q24': {'Definitely false': 0.0, 'Mostly false': 25.0, 'Don\\'t know': 50.0, 'Mostly true': 75.0, 'Definitely true': 100.0}\n}\n\n# Map each column in `df` to the corresponding keys in the `score_mapping` dictionary\ncolumn_to_key = {\n    '1': 'Q1', '2': 'Q2', '3a': 'Q3', '3b': 'Q3', '3c': 'Q3', '3d': 'Q3', '3e': 'Q3', '3f': 'Q3', '3g': 'Q3', '3h': 'Q3', '3i': 'Q3', '3j': 'Q3',\n    '4a': 'Q4 / Q5', '4b': 'Q4 / Q5', '4c': 'Q4 / Q5', '4d': 'Q4 / Q5', '5a': 'Q4 / Q5', '5b': 'Q4 / Q5', '5c': 'Q4 / Q5', '6': 'Q6', '7': 'Q7', '8': 'Q8',\n    '9a': 'Q9 a,d,e,h', '9b': 'Q9 b,c,f,g,i', '9c': 'Q9 b,c,f,g,i', '9d': 'Q9 a,d,e,h', '9e': 'Q9 a,d,e,h', '9f': 'Q9 b,c,f,g,i', '9g': 'Q9 b,c,f,g,i', '9h': 'Q9 a,d,e,h', '9i': 'Q9 b,c,f,g,i',\n    '10': 'Q10', '11a': 'Q11 a,c', '11b': 'Q11 b,d', '11c': 'Q11 a,c', '11d': 'Q11 b,d', '12a': 'Q12', '12b': 'Q12', '12c': 'Q12', '12d': 'Q12',\n    '13a': 'Q13 a-d, f', '13b': 'Q13 a-d, f', '13c': 'Q13 a-d, f', '13d': 'Q13 a-d, f', '13e': 'Q13 e', '13f': 'Q13 a-d, f', '14a': 'Q14', '14b': 'Q14', '14c': 'Q14', '14d': 'Q14', '14e': 'Q14', '14f': 'Q14', '14g': 'Q14', '14h': 'Q14', '14i': 'Q14', '14j': 'Q14', '14k': 'Q14', '14l': 'Q14',\n    '15a': 'Q15', '15b': 'Q15', '15c': 'Q15', '15d': 'Q15', '15e': 'Q15', '15f': 'Q15', '15g': 'Q15', '15h': 'Q15', '16': 'Q16', '16a': 'Q16 a,b', '16b': 'Q16 a,b',\n    '18a': 'Q18 a,c', '18b': 'Q18 b', '18c': 'Q18 a,c', '19a': 'Q19', '19b': 'Q19', '20': 'Q20', '21': 'Q21', '22': 'Q22', '23': 'Q23', '24a': 'Q24', '24b': 'Q24'\n}\n\n# Assign each list of survey items to the corresponding items in the KDQOL survey\nkdqol_domains = {\n    \"PCS\": ['1', '3a', '3b', '3c', '3d', '3e', '3f', '3g', '3h', '3i', '3j', '4a', '4b', '4c', '4d', '7', '8', '11a', '11b', '11c', '11d'],\n    \"MCS\": ['5a', '5b', '5c', '6', '9a', '9b', '9c', '9d', '9e', '9f', '9g', '9h', '9i', '10'],\n    \"SPKD\": ['14a', '14b', '14c', '14d', '14e', '14f', '14g', '14h', '14i', '14j', '14k', '14l'],\n    \"EKD\": ['15a', '15b', '15c', '15d', '15e', '15f', '15g', '15h'],\n    \"BKD\": ['12a', '12b', '12c', '12d']\n}\n\n# Function to map responses to scores\ndef map_responses_to_scores(response, question):\n    mapping = score_mapping.get(question, {})\n    return mapping.get(response, np.nan)\n\n# Create a DataFrame to store whether the score has improved for each domain\nimprove_df = pd.DataFrame(columns=kdqol_domains.keys())\n\n# Group by 'id' and map responses to scores, then check if the score has improved\nfor i, (patient_id, group) in enumerate(df.groupby('id')):\n    scores_df = pd.DataFrame()\n    for domain, columns in kdqol_domains.items():\n        scores = []\n        for col in columns:\n            question = column_to_key.get(col, None)\n            if question:\n                scores.append(group[col].apply(lambda x: map_responses_to_scores(x, question)))\n        scores_df[domain] = pd.concat(scores, axis=1).mean(axis=1, skipna=True)\n    improved = (scores_df.iloc[1] - scores_df.iloc[0]) > 0\n    # Store the result in the DataFrame\n    improve_df.loc[patient_id] = improved\n\n# Save the DataFrame of score improvements to a CSV file\nimprove_df.to_csv(\"improve_df.csv\", index=True)\n```\n\n### Explanation:\n1. **Loading Data**: The DataFrame containing patient responses is loaded.\n2. **Score Mapping**: The provided score mapping dictionary is used to convert responses to scores.\n3. **Domain Assignment**: Each KDQOL domain is assigned its corresponding items.\n4. **Mapping Function**: A function is defined to map responses to scores.\n5. **Improvement Calculation**: For each patient, the average score for each domain is calculated for both surveys, and the improvement is determined by comparing the scores.\n6. **Result Storage**: The results are stored in a DataFrame and saved to a CSV file.\n\nThis code will provide the improvement status of KDQOL domain scores for each patient in the specified DataFrame format."
print(response)

To address the physician's questions regarding the improvement of KDQOL domain scores, we need to follow these steps:

1. **Extract relevant items for each KDQOL domain.**
2. **Map the responses to scores using the provided score mapping dictionary.**
3. **Calculate the average score for each domain.**
4. **Compare the scores from the first and second surveys to determine if there has been an improvement.**

Below is the Python code to achieve this:

```python
import pandas as pd
import numpy as np

# Load the DataFrame containing all the patients' responses
df = pd.read_csv("../eng_base_sub.csv")  # no need to modify the path

# Define the mapping of responses to scores
score_mapping = {
    'Q1': {'Poor': 0.0, 'Fair': 25.0, 'Good': 50.0, 'Very good': 75.0, 'Excellent': 100.0},
    'Q2': {'Much worse now than one year ago': 0.0, 'Somewhat worse now than one year ago': 25.0, 'About the same than one year ago': 50.0, 'Somewhat better now than one year ago': 75.0, 'Much better now than o

In [4]:
import pandas as pd
import numpy as np

# Load the DataFrame containing all the patients' responses
df = pd.read_csv("../eng_base_sub.csv", na_values=[''], keep_default_na=False) # no need to modify the path

# Define the mapping of responses to scores
score_mapping = {
    'Q1': {'Poor': 0.0, 'Fair': 25.0, 'Good': 50.0, 'Very good': 75.0, 'Excellent': 100.0},
    'Q2': {'Much worse now than one year ago': 0.0, 'Somewhat worse now than one year ago': 25.0, 'About the same than one year ago': 50.0, 'Somewhat better now than one year ago': 75.0, 'Much better now than one year ago': 100.0},
    'Q3': {'Yes, limited a lot': 0.0, 'Yes, limited a little': 50.0, 'No, not limited at all': 100.0},
    'Q4 / Q5': {'Yes': 0.0, 'No': 100.0},
    'Q6': {'Extremely': 0.0, 'Quite a bit': 25.0, 'Moderately': 50.0, 'Slightly': 75.0, 'Not at all': 100.0},
    'Q7': {'Very severe': 0.0, 'Severe': 20.0, 'Moderate': 40.0, 'Mild': 60.0, 'Very mild': 80.0, 'None': 100.0},
    'Q8': {'Extremely': 0.0, 'Quite a bit': 25.0, 'Moderately': 50.0, 'A little bit': 75.0, 'Not at all': 100.0},
    'Q9 a,d,e,h': {'None of the time': 0.0, 'A little of the time': 20.0, 'Some of the time': 40.0, 'A good bit of the time': 60.0, 'Most of the time': 80.0, 'All of the time': 100.0},
    'Q9 b,c,f,g,i': {'All of the time': 0.0, 'Most of the time': 20.0, 'A good bit of the time': 40.0, 'Some of the time': 60.0, 'A little of the time': 80.0, 'None of the time': 100.0},
    'Q10': {'All of the time': 0.0, 'Most of the time': 25.0, 'Some of the time': 50.0, 'A little of the time': 75.0, 'None of the time': 100.0},
    'Q11 a,c': {'Definitely true': 0.0, 'Mostly true': 25.0, 'Don\'t know': 50.0, 'Mostly false': 75.0, 'Definitely false': 100.0},
    'Q11 b,d': {'Definitely false': 0.0, 'Mostly false': 25.0, 'Don\'t know': 50.0, 'Mostly true': 75.0, 'Definitely true': 100.0},
    'Q12': {'Definitely true': 0.0, 'Mostly true': 25.0, 'Don\'t know': 50.0, 'Mostly false': 75.0, 'Definitely false': 100.0},
    'Q13 a-d, f': {'All of the time': 0.0, 'Most of the time': 20.0, 'A good bit of the time': 40.0, 'Some of the time': 60.0, 'A little of the time': 80.0, 'None of the time': 100.0},
    'Q13 e': {'None of the time': 0.0, 'A little of the time': 20.0, 'Some of the time': 40.0, 'A good bit of the time': 60.0, 'Most of the time': 80.0, 'All of the time': 100.0},
    'Q14': {'Extremely bothered': 0.0, 'Very much bothered': 25.0, 'Moderately bothered': 50.0, 'Somewhat bothered': 75.0, 'Not at all bothered': 100.0},
    'Q15': {'Extremely bothered': 0.0, 'Very much bothered': 25.0, 'Moderately bothered': 50.0, 'Somewhat bothered': 75.0, 'Not at all bothered': 100.0},
    'Q16': {'No': 0.0, 'Yes': 100.0},
    'Q16 a,b': {'Severe problem': 0.0, 'Very much a problem': 25.0, 'Somewhat of a problem': 50.0, 'A little problem': 75.0, 'Not a problem': 100.0},
    'Q18 a,c': {'All of the time': 0.0, 'Most of the time': 20.0, 'A good bit of the time': 40.0, 'Some of the time': 60.0, 'A little of the time': 80.0, 'None of the time': 100.0},
    'Q18 b': {'None of the time': 0.0, 'A little of the time': 20.0, 'Some of the time': 40.0, 'A good bit of the time': 60.0, 'Most of the time': 80.0, 'All of the time': 100.0},
    'Q19': {'Very dissatisfied': 0.0, 'Somewhat dissatisfied': 33.33, 'Somewhat satisfied': 66.67, 'Very satisfied': 100.0},
    'Q20': {'No': 0.0, 'Yes': 100.0},
    'Q21': {'Yes': 0.0, 'No': 100.0},
    'Q23': {'Very poor': 0.0, 'Poor': 16.67, 'Fair': 33.33, 'Good': 50.0, 'Very good': 66.67, 'Excellent': 83.33, 'The best': 100.0},
    'Q24': {'Definitely false': 0.0, 'Mostly false': 25.0, 'Don\'t know': 50.0, 'Mostly true': 75.0, 'Definitely true': 100.0}
}

# Map each column in `df` to the corresponding keys in the `score_mapping` dictionary
column_to_key = {
    '1': 'Q1', '2': 'Q2', '3a': 'Q3', '3b': 'Q3', '3c': 'Q3', '3d': 'Q3', '3e': 'Q3', '3f': 'Q3', '3g': 'Q3', '3h': 'Q3', '3i': 'Q3', '3j': 'Q3',
    '4a': 'Q4 / Q5', '4b': 'Q4 / Q5', '4c': 'Q4 / Q5', '4d': 'Q4 / Q5', '5a': 'Q4 / Q5', '5b': 'Q4 / Q5', '5c': 'Q4 / Q5', '6': 'Q6', '7': 'Q7', '8': 'Q8',
    '9a': 'Q9 a,d,e,h', '9b': 'Q9 b,c,f,g,i', '9c': 'Q9 b,c,f,g,i', '9d': 'Q9 a,d,e,h', '9e': 'Q9 a,d,e,h', '9f': 'Q9 b,c,f,g,i', '9g': 'Q9 b,c,f,g,i', '9h': 'Q9 a,d,e,h', '9i': 'Q9 b,c,f,g,i',
    '10': 'Q10', '11a': 'Q11 a,c', '11b': 'Q11 b,d', '11c': 'Q11 a,c', '11d': 'Q11 b,d', '12a': 'Q12', '12b': 'Q12', '12c': 'Q12', '12d': 'Q12',
    '13a': 'Q13 a-d, f', '13b': 'Q13 a-d, f', '13c': 'Q13 a-d, f', '13d': 'Q13 a-d, f', '13e': 'Q13 e', '13f': 'Q13 a-d, f', '14a': 'Q14', '14b': 'Q14', '14c': 'Q14', '14d': 'Q14', '14e': 'Q14', '14f': 'Q14', '14g': 'Q14', '14h': 'Q14', '14i': 'Q14', '14j': 'Q14', '14k': 'Q14', '14l': 'Q14',
    '15a': 'Q15', '15b': 'Q15', '15c': 'Q15', '15d': 'Q15', '15e': 'Q15', '15f': 'Q15', '15g': 'Q15', '15h': 'Q15', '16': 'Q16', '16a': 'Q16 a,b', '16b': 'Q16 a,b',
    '18a': 'Q18 a,c', '18b': 'Q18 b', '18c': 'Q18 a,c', '19a': 'Q19', '19b': 'Q19', '20': 'Q20', '21': 'Q21', '22': 'Q22', '23': 'Q23', '24a': 'Q24', '24b': 'Q24'
}

# Assign each list of survey items to the corresponding items in the KDQOL survey
kdqol_domains = {
    "PCS": ['1', '3a', '3b', '3c', '3d', '3e', '3f', '3g', '3h', '3i', '3j', '4a', '4b', '4c', '4d', '7', '8', '11a', '11b', '11c', '11d'],
    "MCS": ['5a', '5b', '5c', '6', '9a', '9b', '9c', '9d', '9e', '9f', '9g', '9h', '9i', '10'],
    "SPKD": ['14a', '14b', '14c', '14d', '14e', '14f', '14g', '14h', '14i', '14j', '14k', '14l'],
    "EKD": ['15a', '15b', '15c', '15d', '15e', '15f', '15g', '15h'],
    "BKD": ['12a', '12b', '12c', '12d']
}

# Function to map responses to scores
def map_responses_to_scores(response, question):
    mapping = score_mapping.get(question, {})
    return mapping.get(response, np.nan)

# Create a DataFrame to store whether the score has improved for each domain
improve_df = pd.DataFrame(columns=kdqol_domains.keys())

# Group by 'id' and map responses to scores, then check if the score has improved
for i, (patient_id, group) in enumerate(df.groupby('id')):
    scores_df = pd.DataFrame()
    for domain, columns in kdqol_domains.items():
        scores = []
        for col in columns:
            question = column_to_key.get(col, None)
            if question:
                scores.append(group[col].apply(lambda x: map_responses_to_scores(x, question)))
        scores_df[domain] = pd.concat(scores, axis=1).mean(axis=1, skipna=True)
    improved = (scores_df.iloc[1] - scores_df.iloc[0]) > 0
    # Store the result in the DataFrame
    improve_df.loc[i] = improved

In [5]:
improve_df

,PCS,MCS,SPKD,EKD,BKD
0,False,False,False,False,True
1,False,False,True,True,True
2,True,False,False,False,False
3,False,False,False,False,False
4,False,True,False,False,False
...,...,...,...,...,...
232,True,True,False,True,True
233,True,False,True,False,False
234,True,True,True,True,True
235,True,False,True,False,True


In [6]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

acc = []
pre = []
rec = []
f1 = []
for col in label.columns:
	true = label[col]
	pred = improve_df[col]
	nan_index = true[true.isna()].index
	true.dropna(inplace=True)
	pred.drop(nan_index, inplace=True)
	true = list(true)
	pred = list(pred)
	acc.append(accuracy_score(true, pred))
	pre.append(precision_score(true, pred))
	rec.append(recall_score(true, pred))
	f1.append(f1_score(true, pred))

In [17]:
label.columns

Index(['PCS', 'MCS', 'SPKD', 'EKD', 'BKD'], dtype='object')

In [7]:
with open('qol_fu_improve_eval.json', 'w', encoding="utf-8") as f:
	json.dump({
		"domain": label.columns.tolist(),
		"accuracy": acc,
		"precision": pre,
		"recall": rec,
		"f1": f1
	}, f, ensure_ascii=False, indent=4)

In [8]:
results = {
    "domain": [
        "PCS",
        "MCS",
        "SPKD",
        "EKD",
        "BKD"
    ],
    "accuracy": [
        1.0,
        1.0,
        1.0,
        1.0,
        1.0
    ],
    "precision": [
        1.0,
        1.0,
        1.0,
        1.0,
        1.0
    ],
    "recall": [
        1.0,
        1.0,
        1.0,
        1.0,
        1.0
    ],
    "f1": [
        1.0,
        1.0,
        1.0,
        1.0,
        1.0
    ]
}

In [9]:
import pandas as pd
results_df = pd.DataFrame(results)
round(results_df, 3)

,domain,accuracy,precision,recall,f1
0,PCS,1.0,1.0,1.0,1.0
1,MCS,1.0,1.0,1.0,1.0
2,SPKD,1.0,1.0,1.0,1.0
3,EKD,1.0,1.0,1.0,1.0
4,BKD,1.0,1.0,1.0,1.0
